In [52]:
import copy
import json
import math
import os
import pathlib
import time
import psycopg2
import requests
from requests.adapters import HTTPAdapter
from requests.auth import HTTPBasicAuth
from urllib3.util.retry import Retry

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import rasterio
from shapely.wkt import loads
from rasterio.mask import mask
import cv2

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

class PGConn:
    def __init__(self, host, port, dbname, user=None, passwd=None):
        self.host = host
        self.port = port
        self.dbname = dbname
        if user is not None:
            self.user = user
        else:
            self.user = ""
        if passwd is not None:
            self.passwd = passwd
        else:
            self.passwd = ""
        self.conn = None

    def connection(self):
        """Return connection to PostgreSQL.  It does not need to be closed
        explicitly.  See the destructor definition below.

        """
        if self.conn is None:
            conn = psycopg2.connect(dbname=self.dbname,
                                    host=self.host,
                                    port=str(self.port),
                                    user=self.user,
                                    password=self.passwd)
            self.conn = conn
            
        return self.conn

pgconn_obj = PGConn(
    "localhost",
    5432,
    "dolr",
    "sameer",
    "swimgood"
)
    
pgconn=pgconn_obj.connection()

In [53]:
query = "select crop_presence, red, green, blue, nir from pilot.cropping_presence_dagdagad as c join pilot.cropping_pattern_dagdagad as p on c.gid = p.gid;"
with pgconn.cursor() as curs:
    curs.execute(query)
    rows = curs.fetchall()

print(rows)

[(0, Decimal('48.686974789915965'), Decimal('44.918067226890756'), Decimal('29.873949579831933'), Decimal('185.35714285714286')), (0, Decimal('48.686974789915965'), Decimal('44.918067226890756'), Decimal('29.873949579831933'), Decimal('185.35714285714286')), (0, Decimal('48.686974789915965'), Decimal('44.918067226890756'), Decimal('29.873949579831933'), Decimal('185.35714285714286')), (0, Decimal('48.686974789915965'), Decimal('44.918067226890756'), Decimal('29.873949579831933'), Decimal('185.35714285714286')), (0, Decimal('48.686974789915965'), Decimal('44.918067226890756'), Decimal('29.873949579831933'), Decimal('185.35714285714286')), (0, Decimal('48.686974789915965'), Decimal('44.918067226890756'), Decimal('29.873949579831933'), Decimal('185.35714285714286')), (1, Decimal('48.686974789915965'), Decimal('44.918067226890756'), Decimal('29.873949579831933'), Decimal('185.35714285714286')), (1, Decimal('48.686974789915965'), Decimal('44.918067226890756'), Decimal('29.873949579831933'),

In [54]:
X = [[float(band) for band in item[1:5]] for item in rows]
for i in range(len(X)):
    rgbn = X[i]
    X[i].append((rgbn[3] - rgbn[0])/(rgbn[3] + rgbn[0]))

X = [[item[-1]] for item in X]
Y = [item[0] for item in rows]

print(len(X))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=8)

model = LogisticRegression()
model.fit(X_train, Y_train)

predictions = model.predict(X_test)
print(classification_report(Y_test, predictions))
print(confusion_matrix(Y_test, predictions))

pgconn.commit()

240
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.58      1.00      0.74        28

    accuracy                           0.58        48
   macro avg       0.29      0.50      0.37        48
weighted avg       0.34      0.58      0.43        48

[[ 0 20]
 [ 0 28]]


/home/sameer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sameer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sameer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
